In [193]:
!python --version

Python 3.12.4


In [194]:
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

import optuna

In [195]:
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 500)

bright_palette = sns.color_palette('bright')
muted_palette = sns.color_palette('muted')
sequential_palette = sns.color_palette('light:dodgerblue_r',9)
alert_palette = sns.color_palette('Reds_r',9)
paired_palette = sns.color_palette('Paired')
#plt.rcParams.update({'figure.dpi' : 240})

import warnings
warnings.filterwarnings("ignore")

from sklearn.cluster import Birch
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.mixture import GaussianMixture
from sklearn.cluster import SpectralClustering, AgglomerativeClustering, KMeans
from kmodes.kprototypes import KPrototypes

from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

#from lazypredict.Supervised import LazyClassifier
from sklearn.model_selection import cross_val_score ,StratifiedKFold, train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder, RobustScaler

le = LabelEncoder()
skf10 = StratifiedKFold(n_splits=10, shuffle=True,random_state=24)
skf5 = StratifiedKFold(n_splits=5, shuffle=True,random_state=24)

from lofo import LOFOImportance, Dataset, plot_importance
from sklearn.feature_selection import RFECV
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

import optuna
from scipy.stats import mode
from itertools import combinations
from tqdm.notebook import tqdm
tqdm.pandas()


In [196]:
test = pd.read_csv('data\\test_x.csv')
train = pd.read_csv('data\\train.csv')

In [197]:
train

,Basvuru Yili,Degerlendirme Puani,Cinsiyet,Dogum Tarihi,Dogum Yeri,Ikametgah Sehri,Universite Adi,Universite Turu,Burslu ise Burs Yuzdesi,Burs Aliyor mu?,Bölüm,Universite Kacinci Sinif,Universite Not Ortalamasi,Daha Once Baska Bir Universiteden Mezun Olmus,Lise Adi,Lise Adi Diger,Lise Sehir,Lise Turu,Lise Bolumu,Lise Bolum Diger,Lise Mezuniyet Notu,Baska Bir Kurumdan Burs Aliyor mu?,Burs Aldigi Baska Kurum,Baska Kurumdan Aldigi Burs Miktari,Anne Egitim Durumu,Anne Calisma Durumu,Anne Sektor,Baba Egitim Durumu,Baba Calisma Durumu,Baba Sektor,Kardes Sayisi,Girisimcilik Kulupleri Tarzi Bir Kulube Uye misiniz?,Uye Oldugunuz Kulubun Ismi,Profesyonel Bir Spor Daliyla Mesgul musunuz?,Spor Dalindaki Rolunuz Nedir?,Aktif olarak bir STK üyesi misiniz?,Hangi STK'nin Uyesisiniz?,Stk Projesine Katildiniz Mi?,Girisimcilikle Ilgili Deneyiminiz Var Mi?,Girisimcilikle Ilgili Deneyiminizi Aciklayabilir misiniz?,Ingilizce Biliyor musunuz?,Ingilizce Seviyeniz?,"Daha Önceden Mezun Olunduysa, Mezun Olunan Üniversite",id
0,2014,52.0,Erkek,4/6/1994,"Altindag, Ankara",Ankara,İHSAN DOĞRAMACI BİLKENT,Özel,100.0,Evet,Bilgisayar Muhendisligi,3,3.50-3,Hayır,Ankara Ataturk Anadolu Lisesi,NaN,Ankara,Anadolu lisesi,MF,NaN,3.50-3,Hayır,NaN,NaN,Üniversite,Hayır,0,Üniversite,Emekli,0,1,Evet,Bilkent IEEE Bilgisayar Toplulugu 2013-2014 Ba...,Evet,Diğer,Hayır,NaN,Evet,Hayır,NaN,NaN,NaN,NaN,0
1,2014,30.0,Erkek,6/11/1993,Üsküdar,İstanbul,İHSAN DOĞRAMACI BİLKENT,Özel,100.0,Hayır,Elektrik Elektronik Mühendisliği,3,3.00-2.50,Hayır,Betül Can Anadolu Lisesi,NaN,Ankara,Anadolu lisesi,MF,NaN,3.00 - 4.00,Hayır,NaN,NaN,Üniversite,Evet,Kamu,Yüksek Lisans / Doktora,Emekli,0,1,Hayır,NaN,Hayır,0,Hayır,NaN,Evet,Evet,Bilkent Cyberparkta bir şirkette Türkiye nin i...,NaN,NaN,NaN,1
2,2014,18.0,Erkek,1/15/1986,Samsun,İstanbul,ULUSLARARASI KIBRIS ÜNİVERSİTESİ,Özel,100.0,Hayır,Finans Yönetimi ve Pazarlama,1,NaN,Hayır,Batem,NaN,Ankara,Meslek lisesi,Elektirk,NaN,3.50-3,Hayır,NaN,NaN,İlkokul Mezunu,Hayır,0,İlkokul Mezunu,Evet,Özel Sektör,6,Evet,Tema,Hayır,0,Hayır,NaN,Hayır,Hayır,NaN,NaN,NaN,NaN,2
3,2014,40.0,Erkek,6/4/1991,Diyarbakır,İstanbul,İSTANBUL ŞEHİR ÜNİVERSİTESİ,Özel,100.0,Evet,Bilgisayar Mühendisliği,3,3.00-2.50,Hayır,Cumhuriyet Fen Lisesi,NaN,Diyarbakır,Fen lisesi,Sayısal,NaN,3.50-3,Hayır,NaN,NaN,İlkokul Mezunu,Hayır,0,Üniversite,Emekli,0,10,Evet,Güzel Konuşma ve yazma kulübü,Hayır,0,Hayır,NaN,Evet,Hayır,NaN,NaN,NaN,NaN,3
4,2014,24.0,Erkek,2 Kasim 1992,Ankara/Altındağ,Ankara,TURGUT ÖZAL ÜNİVERSİTESİ,Özel,100.0,Evet,Siyaset Bilimi ve Uluslararası ilişkiler,NaN,3.00-2.50,Hayır,Samanyolu Lisesi,NaN,Ankara,Özel lisesi,TM,NaN,3.00 - 4.00,Hayır,NaN,NaN,İlkokul Mezunu,Hayır,0,Üniversite,Emekli,0,1,Hayır,NaN,Hayır,0,Hayır,NaN,Hayır,Hayır,NaN,NaN,NaN,NaN,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65120,2022,0.0,Erkek,18.06.2003 00:00,İstanbul,İstanbul,İstanbul Aydın Üniversitesi,Özel,NaN,Evet,Muhasebe,2,0 - 1.79,NaN,BAĞCILAR EDİP İPLİK MESLEKİ VE TEKNİK ANADOLU ...,NaN,İstanbul,Devlet,Eşit Ağırlık,NaN,50 - 74,Hayır,-,-,İlkokul,Hayır,-,Ortaokul,Evet,Özel Sektör,4.0,Hayır,NaN,Hayır,-,Hayır,-,NaN,Hayır,-,Hayır,NaN,NaN,65120
65121,2022,0.0,Erkek,18.04.2002 00:00,Hatay,Muğla,Muğla Sıtkı Koçman Üniversitesi,Devlet,NaN,Hayır,Turizm İşletmeciliği,1,Hazırlığım,NaN,Ahmet Vefik paşa Anadolu lisesi,NaN,Bursa,Devlet,Eşit Ağırlık,NaN,75 - 100,Hayır,-,-,Lise,Hayır,-,Lise,Evet,Özel Sektör,4.0,Hayır,NaN,Hayır,-,Hayır,-,NaN,Hayır,-,Hayır,NaN,NaN,65121
65122,2022,0.0,Erkek,23.10.1999 00:00,Mersin,Mersin,Süleyman Demirel Üniversitesi,Devlet,NaN,Hayır,Otomotiv Mühendisliği,4,0 - 1.79,NaN,Açık lise,NaN,Mersin,Devlet,Sayısal,NaN,75 - 100,Evet,Kyk,500₺ - 999₺,Lise,Hayır,-,Lise,Hayır,-,3.0,Hayır,NaN,Hayır,-,Hayır,-,NaN,Hayır,-,Hayır,NaN,NaN,65122
65123,2022,0.0,Erkek,17.11.2000 00:00,Samsun,Samsun,Karabük Üniversitesi,Devlet,NaN,Hayır,Sanat Tarihi,3,0 - 1.79,NaN,Samsun Terme K

In [198]:
test

,Basvuru Yili,Cinsiyet,Dogum Tarihi,Dogum Yeri,Ikametgah Sehri,Universite Adi,Universite Turu,Burslu ise Burs Yuzdesi,Burs Aliyor mu?,Bölüm,Universite Kacinci Sinif,Universite Not Ortalamasi,Daha Once Baska Bir Universiteden Mezun Olmus,Lise Adi,Lise Adi Diger,Lise Sehir,Lise Turu,Lise Bolumu,Lise Bolum Diger,Lise Mezuniyet Notu,Baska Bir Kurumdan Burs Aliyor mu?,Burs Aldigi Baska Kurum,Baska Kurumdan Aldigi Burs Miktari,Anne Egitim Durumu,Anne Calisma Durumu,Anne Sektor,Baba Egitim Durumu,Baba Calisma Durumu,Baba Sektor,Kardes Sayisi,Girisimcilik Kulupleri Tarzi Bir Kulube Uye misiniz?,Uye Oldugunuz Kulubun Ismi,Profesyonel Bir Spor Daliyla Mesgul musunuz?,Spor Dalindaki Rolunuz Nedir?,Aktif olarak bir STK üyesi misiniz?,Hangi STK'nin Uyesisiniz?,Stk Projesine Katildiniz Mi?,Girisimcilikle Ilgili Deneyiminiz Var Mi?,Girisimcilikle Ilgili Deneyiminizi Aciklayabilir misiniz?,Ingilizce Biliyor musunuz?,Ingilizce Seviyeniz?,"Daha Önceden Mezun Olunduysa, Mezun Olunan Üniversite",id
0,2023,Kadın,21.06.2002 00:00,Amasya,Amasya,Munzur Üniversitesi,Devlet,NaN,Hayır,Sosyal Hizmet,1,2.50 - 2.99,NaN,Amasya Suluova Şehit Muharrem Saygun Anadolu L...,NaN,Amasya,Devlet,Eşit Ağırlık,NaN,50 - 74,Hayır,-,-,İlkokul,Evet,Özel Sektör,Ortaokul,Evet,Diğer,2.0,Hayır,NaN,Evet,Takım Oyuncusu,Evet,Kızılay da gönüllü dağıtım,NaN,Hayır,-,Evet,NaN,NaN,0
1,2023,Kadın,11.05.2004 00:00,Konya,Konya,Hacettepe Üniversitesi,Devlet,NaN,Hayır,Uluslararası İlişkiler,1,Hazırlığım,NaN,Açık Öğretim Lisesi,NaN,Konya,Devlet,Eşit Ağırlık,NaN,75 - 100,Hayır,-,-,İlkokul,Hayır,-,İlkokul,Evet,Özel Sektör,3.0,Hayır,NaN,Evet,Bireysel Spor,Hayır,-,NaN,Hayır,-,Evet,NaN,NaN,1
2,2023,Erkek,24.04.2002 00:00,İstanbul,İstanbul,Kapadokya Üniversitesi,Özel,NaN,Evet,Diğer,1,Hazırlığım,NaN,Özel Aşama anadolu lisesi,NaN,İstanbul,Özel,Eşit Ağırlık,NaN,50 - 74,Evet,KYK,1000₺ ve üstü,Yüksek Lisans,Hayır,-,Üniversite,Hayır,-,0.0,Hayır,NaN,Hayır,Bireysel Spor,Hayır,-,NaN,Hayır,-,Evet,NaN,NaN,2
3,2023,Erkek,11.03.2003 00:00,Mardin,Mardin,Mardin Artuklu Üniversitesi,Devlet,NaN,Hayır,İslami İlimler,1,Hazırlığım,NaN,Özcan yıldız AİHL,NaN,Mardin,Devlet,Sözel,NaN,75 - 100,Hayır,-,-,İlkokul,Hayır,-,Lise,Evet,Özel Sektör,4.0,Hayır,NaN,Hayır,-,Evet,Lise gezilerde liderlik yapıyordum,NaN,Hayır,-,Hayır,NaN,NaN,3
4,2023,Erkek,26.01.2002 00:00,Samsun,İstanbul,Boğaziçi Üniversitesi,Devlet,NaN,Hayır,İşletme,3,3.00 - 3.49,NaN,Özel Bafra Açı Fen Lisesi,NaN,Samsun,Özel,Eşit Ağırlık,NaN,75 - 100,Evet,Toplum Gönüllüleri Vakfı,500₺ - 999₺,Üniversite,Evet,Kamu,Üniversite,Evet,Kamu,1.0,Hayır,NaN,Hayır,-,Evet,"Toplum Gönüllüleri Vakfı'nda ""Yol Arkadaşım"" P...",NaN,Evet,"12 aydır, yolculuğuna bir yıl önce başlamış ol...",Evet,NaN,NaN,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11044,2023,Kadın,3.01.2002 00:00,Yozgat,Ankara,Gazi Üniversitesi,Devlet,NaN,Hayır,Endüstri Mühendisliği,2,1.80 - 2.49,NaN,Yozgat Şehitler Fen Lisesi,NaN,Yozgat,Devlet,Sayısal,NaN,75 - 100,Evet,KYK,1000₺ ve üstü,İlkokul,Hayır,-,Ortaokul,Hayır,-,0.0,Evet,NaN,Hayır,-,Evet,TOG Vakfı'nda yaklaşık bir yıldır hem aktif gö...,NaN,Hayır,-,Evet,NaN,NaN,11044
11045,2023,Kadın,9.09.2001 00:00,Konya,Konya,Niğde Ömer Halisdemir Üniversitesi,Devlet,NaN,Hayır,Gıda Mühendisliği,3,3.00 - 3.49,NaN,Prof. Dr. Ömer Dinçer Kız Anadolu İmam Hatip L...,NaN,Konya,Devlet,Sayısal,NaN,75 - 100,Hayır,-,-,İlkokul,Hayır,-,İlkokul,Evet,Kamu,3.0,Evet,NaN,Hayır,-,Evet,Konyada düzenlenen 5. İslami dayanışma oyunlar...,NaN,Hayır,-,Hayır,NaN,NaN,11045
11046,2023,Erkek,8.06.2004 00:00,Ankara,Ankara,Karabük Üniversitesi,Devlet,NaN,Hayır,Elektrik-Elektronik Mühendisliği,1,Hazırlığım,NaN,Nokta tek koleji,NaN,Ankara,Özel,Sayısal,NaN,75 - 100,Hayır,-,-,İlkokul,Hayır,-,Üniversite,Evet,Diğer,2.0,Evet,NaN,Evet,Takım Oyuncusu,Evet,Hayvanlara yardım adı altında hayvanlara yardı...,NaN,Hayır,-,Evet,NaN,NaN,11046
11047,2023,Kadın,16.10.2001 00:00,Adıyaman,İstan

In [199]:
print(train.isna().sum().sort_values(ascending=False))

Daha Önceden Mezun Olunduysa, Mezun Olunan Üniversite        64706
Lise Bolum Diger                                             63599
Lise Adi Diger                                               60392
Girisimcilikle Ilgili Deneyiminizi Aciklayabilir misiniz?    50526
Hangi STK'nin Uyesisiniz?                                    49518
Uye Oldugunuz Kulubun Ismi                                   48691
Baska Kurumdan Aldigi Burs Miktari                           44162
Burs Aldigi Baska Kurum                                      43977
Burslu ise Burs Yuzdesi                                      42685
Daha Once Baska Bir Universiteden Mezun Olmus                37345
Ingilizce Seviyeniz?                                         36533
Anne Sektor                                                  34537
Spor Dalindaki Rolunuz Nedir?                                32275
Stk Projesine Katildiniz Mi?                                 28308
Baba Sektor                                                  2

In [200]:
train["Girisimcilik Kulupleri Tarzi Bir Kulube Uye misiniz?"].value_counts("Evet",dropna=False)


Girisimcilik Kulupleri Tarzi Bir Kulube Uye misiniz?
Hayır    0.517620
Evet     0.454296
NaN      0.028084
Name: proportion, dtype: float64

In [201]:
print(train.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65125 entries, 0 to 65124
Data columns (total 44 columns):
 #   Column                                                     Non-Null Count  Dtype  
---  ------                                                     --------------  -----  
 0   Basvuru Yili                                               65125 non-null  int64  
 1   Degerlendirme Puani                                        65124 non-null  float64
 2   Cinsiyet                                                   64956 non-null  object 
 3   Dogum Tarihi                                               64948 non-null  object 
 4   Dogum Yeri                                                 64334 non-null  object 
 5   Ikametgah Sehri                                            63088 non-null  object 
 6   Universite Adi                                             64993 non-null  object 
 7   Universite Turu                                            64870 non-null  object 
 8   Burslu

In [202]:
print(test.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11049 entries, 0 to 11048
Data columns (total 43 columns):
 #   Column                                                     Non-Null Count  Dtype  
---  ------                                                     --------------  -----  
 0   Basvuru Yili                                               11049 non-null  int64  
 1   Cinsiyet                                                   11049 non-null  object 
 2   Dogum Tarihi                                               11049 non-null  object 
 3   Dogum Yeri                                                 11049 non-null  object 
 4   Ikametgah Sehri                                            11049 non-null  object 
 5   Universite Adi                                             11049 non-null  object 
 6   Universite Turu                                            11049 non-null  object 
 7   Burslu ise Burs Yuzdesi                                    0 non-null      float64
 8   Burs A

In [203]:
print(test.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11049 entries, 0 to 11048
Data columns (total 43 columns):
 #   Column                                                     Non-Null Count  Dtype  
---  ------                                                     --------------  -----  
 0   Basvuru Yili                                               11049 non-null  int64  
 1   Cinsiyet                                                   11049 non-null  object 
 2   Dogum Tarihi                                               11049 non-null  object 
 3   Dogum Yeri                                                 11049 non-null  object 
 4   Ikametgah Sehri                                            11049 non-null  object 
 5   Universite Adi                                             11049 non-null  object 
 6   Universite Turu                                            11049 non-null  object 
 7   Burslu ise Burs Yuzdesi                                    0 non-null      float64
 8   Burs A

In [204]:
nan_indices = train.loc[train['Universite Adi'].isna()].index

In [205]:
train['Basvuru Yili'].value_counts().sort_index()

Basvuru Yili
2014     6454
2015     7804
2016     5213
2017    10325
2018     1105
2019     7248
2020     7760
2021    11594
2022     7622
Name: count, dtype: int64

In [206]:
train['Dogum Tarihi']

0                4/6/1994
1               6/11/1993
2               1/15/1986
3                6/4/1991
4            2 Kasim 1992
               ...       
65120    18.06.2003 00:00
65121    18.04.2002 00:00
65122    23.10.1999 00:00
65123    17.11.2000 00:00
65124    27.04.2000 00:00
Name: Dogum Tarihi, Length: 65125, dtype: object

In [207]:
# Extract year from 'Dogum Tarihi' column
def extract_year(date_string):
    try:
        # For dates in the format "DD.MM.YYYY 00:00"
        if '.' in date_string and ':' in date_string:
            return int(date_string.split('.')[-1].split()[0])
        # For dates in the format "M/D/YYYY"
        elif '/' in date_string:
            return int(date_string.split('/')[-1])
        # For dates in the format "D Month YYYY"
        else:
            return int(date_string.split()[-1])
    except:
        return None

# Apply the function to create a new column with birth years
Year_yusuf = train['Dogum Tarihi'].apply(extract_year)

# Display the first few rows of the new column
print(train['Dogum Tarihi'],Year_yusuf)

# Check for any None values (parsing errors)
print("\nNumber of None values:", Year_yusuf.isna().sum())

0                4/6/1994
1               6/11/1993
2               1/15/1986
3                6/4/1991
4            2 Kasim 1992
               ...       
65120    18.06.2003 00:00
65121    18.04.2002 00:00
65122    23.10.1999 00:00
65123    17.11.2000 00:00
65124    27.04.2000 00:00
Name: Dogum Tarihi, Length: 65125, dtype: object 0        1994.0
1        1993.0
2        1986.0
3        1991.0
4        1992.0
          ...  
65120    2003.0
65121    2002.0
65122    1999.0
65123    2000.0
65124    2000.0
Name: Dogum Tarihi, Length: 65125, dtype: float64

Number of None values: 30938


In [208]:
train

,Basvuru Yili,Degerlendirme Puani,Cinsiyet,Dogum Tarihi,Dogum Yeri,Ikametgah Sehri,Universite Adi,Universite Turu,Burslu ise Burs Yuzdesi,Burs Aliyor mu?,Bölüm,Universite Kacinci Sinif,Universite Not Ortalamasi,Daha Once Baska Bir Universiteden Mezun Olmus,Lise Adi,Lise Adi Diger,Lise Sehir,Lise Turu,Lise Bolumu,Lise Bolum Diger,Lise Mezuniyet Notu,Baska Bir Kurumdan Burs Aliyor mu?,Burs Aldigi Baska Kurum,Baska Kurumdan Aldigi Burs Miktari,Anne Egitim Durumu,Anne Calisma Durumu,Anne Sektor,Baba Egitim Durumu,Baba Calisma Durumu,Baba Sektor,Kardes Sayisi,Girisimcilik Kulupleri Tarzi Bir Kulube Uye misiniz?,Uye Oldugunuz Kulubun Ismi,Profesyonel Bir Spor Daliyla Mesgul musunuz?,Spor Dalindaki Rolunuz Nedir?,Aktif olarak bir STK üyesi misiniz?,Hangi STK'nin Uyesisiniz?,Stk Projesine Katildiniz Mi?,Girisimcilikle Ilgili Deneyiminiz Var Mi?,Girisimcilikle Ilgili Deneyiminizi Aciklayabilir misiniz?,Ingilizce Biliyor musunuz?,Ingilizce Seviyeniz?,"Daha Önceden Mezun Olunduysa, Mezun Olunan Üniversite",id
0,2014,52.0,Erkek,4/6/1994,"Altindag, Ankara",Ankara,İHSAN DOĞRAMACI BİLKENT,Özel,100.0,Evet,Bilgisayar Muhendisligi,3,3.50-3,Hayır,Ankara Ataturk Anadolu Lisesi,NaN,Ankara,Anadolu lisesi,MF,NaN,3.50-3,Hayır,NaN,NaN,Üniversite,Hayır,0,Üniversite,Emekli,0,1,Evet,Bilkent IEEE Bilgisayar Toplulugu 2013-2014 Ba...,Evet,Diğer,Hayır,NaN,Evet,Hayır,NaN,NaN,NaN,NaN,0
1,2014,30.0,Erkek,6/11/1993,Üsküdar,İstanbul,İHSAN DOĞRAMACI BİLKENT,Özel,100.0,Hayır,Elektrik Elektronik Mühendisliği,3,3.00-2.50,Hayır,Betül Can Anadolu Lisesi,NaN,Ankara,Anadolu lisesi,MF,NaN,3.00 - 4.00,Hayır,NaN,NaN,Üniversite,Evet,Kamu,Yüksek Lisans / Doktora,Emekli,0,1,Hayır,NaN,Hayır,0,Hayır,NaN,Evet,Evet,Bilkent Cyberparkta bir şirkette Türkiye nin i...,NaN,NaN,NaN,1
2,2014,18.0,Erkek,1/15/1986,Samsun,İstanbul,ULUSLARARASI KIBRIS ÜNİVERSİTESİ,Özel,100.0,Hayır,Finans Yönetimi ve Pazarlama,1,NaN,Hayır,Batem,NaN,Ankara,Meslek lisesi,Elektirk,NaN,3.50-3,Hayır,NaN,NaN,İlkokul Mezunu,Hayır,0,İlkokul Mezunu,Evet,Özel Sektör,6,Evet,Tema,Hayır,0,Hayır,NaN,Hayır,Hayır,NaN,NaN,NaN,NaN,2
3,2014,40.0,Erkek,6/4/1991,Diyarbakır,İstanbul,İSTANBUL ŞEHİR ÜNİVERSİTESİ,Özel,100.0,Evet,Bilgisayar Mühendisliği,3,3.00-2.50,Hayır,Cumhuriyet Fen Lisesi,NaN,Diyarbakır,Fen lisesi,Sayısal,NaN,3.50-3,Hayır,NaN,NaN,İlkokul Mezunu,Hayır,0,Üniversite,Emekli,0,10,Evet,Güzel Konuşma ve yazma kulübü,Hayır,0,Hayır,NaN,Evet,Hayır,NaN,NaN,NaN,NaN,3
4,2014,24.0,Erkek,2 Kasim 1992,Ankara/Altındağ,Ankara,TURGUT ÖZAL ÜNİVERSİTESİ,Özel,100.0,Evet,Siyaset Bilimi ve Uluslararası ilişkiler,NaN,3.00-2.50,Hayır,Samanyolu Lisesi,NaN,Ankara,Özel lisesi,TM,NaN,3.00 - 4.00,Hayır,NaN,NaN,İlkokul Mezunu,Hayır,0,Üniversite,Emekli,0,1,Hayır,NaN,Hayır,0,Hayır,NaN,Hayır,Hayır,NaN,NaN,NaN,NaN,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65120,2022,0.0,Erkek,18.06.2003 00:00,İstanbul,İstanbul,İstanbul Aydın Üniversitesi,Özel,NaN,Evet,Muhasebe,2,0 - 1.79,NaN,BAĞCILAR EDİP İPLİK MESLEKİ VE TEKNİK ANADOLU ...,NaN,İstanbul,Devlet,Eşit Ağırlık,NaN,50 - 74,Hayır,-,-,İlkokul,Hayır,-,Ortaokul,Evet,Özel Sektör,4.0,Hayır,NaN,Hayır,-,Hayır,-,NaN,Hayır,-,Hayır,NaN,NaN,65120
65121,2022,0.0,Erkek,18.04.2002 00:00,Hatay,Muğla,Muğla Sıtkı Koçman Üniversitesi,Devlet,NaN,Hayır,Turizm İşletmeciliği,1,Hazırlığım,NaN,Ahmet Vefik paşa Anadolu lisesi,NaN,Bursa,Devlet,Eşit Ağırlık,NaN,75 - 100,Hayır,-,-,Lise,Hayır,-,Lise,Evet,Özel Sektör,4.0,Hayır,NaN,Hayır,-,Hayır,-,NaN,Hayır,-,Hayır,NaN,NaN,65121
65122,2022,0.0,Erkek,23.10.1999 00:00,Mersin,Mersin,Süleyman Demirel Üniversitesi,Devlet,NaN,Hayır,Otomotiv Mühendisliği,4,0 - 1.79,NaN,Açık lise,NaN,Mersin,Devlet,Sayısal,NaN,75 - 100,Evet,Kyk,500₺ - 999₺,Lise,Hayır,-,Lise,Hayır,-,3.0,Hayır,NaN,Hayır,-,Hayır,-,NaN,Hayır,-,Hayır,NaN,NaN,65122
65123,2022,0.0,Erkek,17.11.2000 00:00,Samsun,Samsun,Karabük Üniversitesi,Devlet,NaN,Hayır,Sanat Tarihi,3,0 - 1.79,NaN,Samsun Terme K

In [209]:
import pandas as pd
from dateutil import parser
from datetime import datetime
import re
# Mapping of Turkish month names to English
turkish_to_english_months = {
    "Ocak": "January",
    "Şubat": "February",
    "Mart": "March",
    "Nisan": "April",
    "Mayıs": "May",
    "Haziran": "June",
    "Temmuz": "July",
    "Ağustos": "August",
    "Eylül": "September",
    "Ekim": "October",
    "Kasım": "November",
    "Aralık": "December"
}

def normalize_turkish_months(date_str):
    for turkish_month, english_month in turkish_to_english_months.items():
        date_str = re.sub(r'\b' + turkish_month + r'\b', english_month, date_str)
    return date_str

def extract_year(date_str):
    try:
        # Normalize the date string for Turkish month names
        date_str = normalize_turkish_months(date_str)
        
        # Parse the date string using dateutil.parser
        parsed_date = parser.parse(date_str, fuzzy=True, dayfirst=False)
        
        # Extract the year from the parsed date
        return parsed_date.year
    except (ValueError, TypeError):
        # Return None if parsing fails
        return None


# Apply the extract_year function to the 'date' column
Year_kadir = train['Dogum Tarihi'].apply(extract_year)

# Filter out rows where the year could not be extracted
# filtered_df = df.dropna(subset=['year'])

# Display the result
print(Year_kadir.isna().sum())
print(Year_kadir.isna().index)

1819
RangeIndex(start=0, stop=65125, step=1)


In [210]:
Year_kadir

0        1994.0
1        1993.0
2        1986.0
3        1991.0
4        1992.0
          ...  
65120    2003.0
65121    2002.0
65122    1999.0
65123    2000.0
65124    2000.0
Name: Dogum Tarihi, Length: 65125, dtype: float64

In [211]:
Year_yusuf = train['Dogum Tarihi'].to_string()


len(Year_yusuf)

Year_yusuf

'0                   4/6/1994\n1                  6/11/1993\n2                  1/15/1986\n3                   6/4/1991\n4               2 Kasim 1992\n5                  4/14/1992\n6                 10/13/1994\n7                  3/18/1993\n8             29 Aralik 1991\n9                  4/25/1991\n10                 7/20/1995\n11                 9/27/1993\n12                 3/23/1993\n13                  8/9/1992\n14                  3/5/1992\n15                 4/16/1992\n16                 7/15/1992\n17              8 Mayis 1995\n18                 7/27/1995\n19                 7/24/1995\n20              7 Mayis 1994\n21                 8/15/1995\n22                 1/27/1989\n23                 10/5/1992\n24                  4/9/1994\n25                  6/9/1993\n26             31 Mayis 1992\n27                  2/8/1995\n28                 1/23/1994\n29                 7/23/1994\n30                 4/23/1995\n31                 4/24/1992\n32                 9/24/1993\n33       

In [212]:
#Year_yusuf.isna().sum()

In [213]:
import pandas as pd
from dateutil import parser
from datetime import datetime
import re

# Mapping of Turkish month names to English
turkish_to_english_months = {
    "Ocak": "January",
    "Şubat": "February",
    "Subat": "February",
    "Mart": "March",
    "Nisan": "April",
    "Mayıs": "May",
    "Mayis": "May",
    "Haziran": "June",
    "Temmuz": "July",
    "Ağustos": "August",
    "Agustos": "August",
    "Eylül": "September",
    "Eylul": "September",
    "Ekim": "October",
    "Kasım": "November",
    "Kasim": "November",
    "Aralık": "December",
    "Aralik": "December"
}

def normalize_turkish_months(date_str):
    for turkish_month, english_month in turkish_to_english_months.items():
        date_str = re.sub(r'\b' + turkish_month + r'\b', english_month, date_str)
    return date_str

def extract_year(date_str):
    if not isinstance(date_str, str):
        return None  # Return None if the input is not a string
    try:
        # Normalize the date string for Turkish month names
        date_str = normalize_turkish_months(date_str)
        
        # Parse the date string using dateutil.parser
        parsed_date = parser.parse(date_str, fuzzy=True, dayfirst=False)
        
        # Extract the year from the parsed date
        return parsed_date.year
    except (ValueError, TypeError):
        # If parsing fails, try extracting the year directly from the string
        year_match = re.search(r'(\d{4})$', date_str)
        if year_match:
            return int(year_match.group(1))
        return None

# Apply the extract_year function to the 'date' column
Year_kadir_2 = train['Dogum Tarihi'].apply(extract_year)

# Identify and print rows with NaN values
nan_indices = train[Year_kadir_2.isna()].index
if not nan_indices.empty:
    print("Rows with NaN values and their indices:")
    for idx in nan_indices:
        print(f"Index: {idx}, Date: {train.loc[idx, 'Dogum Tarihi']}")

# Filter out rows where the year could not be extracted
filtered_df = Year_kadir_2.dropna()

# Display the filtered DataFrame
print("\nFiltered DataFrame with valid years:")
# print(filtered_df)
print(np.sort(Year_kadir_2.unique()));
print(Year_kadir_2.isna().sum())

for i in range(194, 201):
    Year_kadir_2 = Year_kadir_2.replace(i, i+1800)


Rows with NaN values and their indices:
Index: 3206, Date: nan
Index: 3331, Date: 06/91/198_
Index: 3897, Date: nan
Index: 4006, Date: nan
Index: 4032, Date: nan
Index: 4370, Date: nan
Index: 4442, Date: nan
Index: 4648, Date: nan
Index: 4707, Date: nan
Index: 4738, Date: nan
Index: 5055, Date: nan
Index: 5762, Date: nan
Index: 5861, Date: nan
Index: 5862, Date: nan
Index: 6100, Date: nan
Index: 6104, Date: nan
Index: 6165, Date: nan
Index: 7979, Date: 19/91/995_
Index: 8434, Date: nan
Index: 8616, Date: nan
Index: 8641, Date: nan
Index: 8736, Date: nan
Index: 8737, Date: nan
Index: 8799, Date: nan
Index: 8930, Date: nan
Index: 8990, Date: nan
Index: 9329, Date: nan
Index: 9984, Date: nan
Index: 9991, Date: nan
Index: 10009, Date: nan
Index: 10066, Date: nan
Index: 10067, Date: nan
Index: 10077, Date: nan
Index: 10233, Date: nan
Index: 10247, Date: nan
Index: 10261, Date: nan
Index: 26567, Date: 99/44/____
Index: 26608, Date: nan
Index: 27060, Date: 11/__/____
Index: 27207, Date: nan
I

In [214]:
print(Year_kadir_2.value_counts()[Year_kadir_2.value_counts()>10]);
print(Year_kadir_2.isna().sum())

Dogum Tarihi
2000.0    7194
1999.0    6698
1998.0    6624
1995.0    6301
1996.0    6159
1997.0    6125
2001.0    5675
1994.0    5399
1993.0    4165
2002.0    3212
1992.0    2585
2070.0    1661
1991.0    1397
2003.0     737
1990.0     601
1989.0      81
1970.0      55
1988.0      49
1987.0      32
2016.0      24
2004.0      23
1986.0      21
2017.0      11
Name: count, dtype: int64
192


In [215]:
import numpy as np

# Replace 194 and 196 with NaN in Year_kadir_2

# Print sorted unique values to verify the change
print(np.sort(Year_kadir_2.unique()))
len(Year_kadir_2)
print(Year_kadir_2.isna().sum())


[ 328.  808.  989.  993.  996.  998. 1155. 1194. 1195. 1196. 1197. 1198.
 1199. 1204. 1262. 1888. 1895. 1896. 1929. 1933. 1944. 1970. 1972. 1976.
 1977. 1979. 1980. 1981. 1982. 1983. 1984. 1985. 1986. 1987. 1988. 1989.
 1990. 1991. 1992. 1993. 1994. 1995. 1996. 1997. 1998. 1999. 2000. 2001.
 2002. 2003. 2004. 2005. 2006. 2009. 2011. 2015. 2016. 2017. 2020. 2021.
 2070. 2996. 5778. 6123. 7199. 9590. 9691. 9916. 9941. 9951.   nan]
192


In [216]:
value_counts = Year_kadir_2.value_counts()

# Step 2: Identify values with counts less than 10
values_to_replace = value_counts[value_counts < 10].index

# Step 3: Replace these values with NaN
Year_kadir_2 = Year_kadir_2.apply(lambda x: np.nan if x in values_to_replace else x)

print(Year_kadir_2)
print(Year_kadir_2.isna().sum())
print(np.sort(Year_kadir_2.unique()))

0        1994.0
1        1993.0
2        1986.0
3        1991.0
4        1992.0
          ...  
65120    2003.0
65121    2002.0
65122    1999.0
65123    2000.0
65124    2000.0
Name: Dogum Tarihi, Length: 65125, dtype: float64
296
[1970. 1986. 1987. 1988. 1989. 1990. 1991. 1992. 1993. 1994. 1995. 1996.
 1997. 1998. 1999. 2000. 2001. 2002. 2003. 2004. 2016. 2017. 2070.   nan]


In [217]:
value_counts

Dogum Tarihi
2000.0    7194
1999.0    6698
1998.0    6624
1995.0    6301
1996.0    6159
1997.0    6125
2001.0    5675
1994.0    5399
1993.0    4165
2002.0    3212
1992.0    2585
2070.0    1661
1991.0    1397
2003.0     737
1990.0     601
1989.0      81
1970.0      55
1988.0      49
1987.0      32
2016.0      24
2004.0      23
1986.0      21
2017.0      11
2005.0       9
2020.0       9
1984.0       7
1985.0       6
1197.0       6
1983.0       5
1976.0       5
1196.0       4
1980.0       4
1977.0       3
1199.0       2
9691.0       2
2011.0       2
1979.0       2
1195.0       2
996.0        2
998.0        2
993.0        2
9951.0       2
7199.0       1
1155.0       1
9590.0       1
1198.0       1
989.0        1
6123.0       1
2009.0       1
808.0        1
1204.0       1
2006.0       1
328.0        1
5778.0       1
2996.0       1
1896.0       1
9916.0       1
1888.0       1
1929.0       1
1944.0       1
1895.0       1
1262.0       1
1933.0       1
1194.0       1
2015.0       1
1972.0      

In [218]:
train["Dogum Yeri"].value_counts().head(81)

Dogum Yeri
İstanbul          8682
Ankara            4045
İzmir             2170
Istanbul          1894
Adana             1810
Bursa             1809
Mersin            1447
Konya             1436
Diyarbakır        1377
Hatay             1326
Gaziantep         1310
Antalya           1174
Kayseri           1141
Van               1099
Şanlıurfa         1082
Mardin             981
Kocaeli            956
Malatya            932
Erzurum            900
Samsun             899
Manisa             847
Denizli            717
Kahramanmaraş      623
Sivas              620
Trabzon            601
Ağrı               598
Batman             597
Balıkesir          595
Ordu               549
Eskişehir          543
Izmir              534
Adıyaman           528
Aydın              510
Sakarya            500
Tokat              492
Çorum              480
Zonguldak          435
Elazığ             427
Yozgat             424
Diğer              405
Şırnak             392
Tekirdağ           389
Muş                385


In [219]:
train["Dogum Yeri"].value_counts().head(81).sum()

57704

In [220]:
def standardize_general(location):

    if 'istanbul'  in location.lower():  # Convert to lowercase to handle case-insensitivity
        return 'İstanbul'
    if 'i̇stanbul'  in location.lower():  # Convert to lowercase to handle case-insensitivity
        return 'İstanbul'
    elif 'ıstanbul' in location.lower():
        return 'İstanbul'
    
    elif 'ankara'  in location.lower():  # Convert to lowercase to handle case-insensitivity
        return 'Ankara'
    elif 'altındag'  in location.lower():  # Convert to lowercase to handle case-insensitivity
        return 'Ankara'
    elif 'altindag'  in location.lower():  # Convert to lowercase to handle case-insensitivity
        return 'Ankara'
    elif 'altındağ'  in location.lower():  # Convert to lowercase to handle case-insensitivity
        return 'Ankara'
    elif 'altindağ'  in location.lower():  # Convert to lowercase to handle case-insensitivity
        return 'Ankara'
    
    elif 'izmir'  in location.lower():  # Convert to lowercase to handle case-insensitivity
        return 'İzmir'
    elif 'ızmir'  in location.lower():  # Convert to lowercase to handle case-insensitivity
        return 'İzmir'
    
    elif 'adana'  in location.lower():  # Convert to lowercase to handle case-insensitivity
        return 'Adana'
    
    elif 'bursa'  in location.lower():  # Convert to lowercase to handle case-insensitivity
        return 'Bursa'
    elif 'konya'  in location.lower():  # Convert to lowercase to handle case-insensitivity
        return 'Konya'
    elif 'mersin'  in location.lower():  # Convert to lowercase to handle case-insensitivity
        return 'Mersin'
    elif 'trabzon'  in location.lower():  # Convert to lowercase to handle case-insensitivity
        return 'Trabzon'
    elif 'kocaeli'  in location.lower():  # Convert to lowercase to handle case-insensitivity
        return 'Kocaeli'
    
    elif 'k.maraş'  in location.lower():  # Convert to lowercase to handle case-insensitivity
        return 'Kahramanmaraş'
    
    else:
        return location.capitalize()


In [221]:
train["Dogum Yeri"] = train["Dogum Yeri"].astype(str).apply(standardize_general)
train["Dogum Yeri"].value_counts().head(81).sum()

60569

In [222]:
train["Dogum Yeri"].value_counts().head(81)

Dogum Yeri
İstanbul          11073
Ankara             4309
İzmir              2767
Bursa              1931
Adana              1914
Konya              1514
Mersin             1472
Diyarbakır         1403
Hatay              1333
Gaziantep          1330
Antalya            1199
Kayseri            1152
Van                1122
Şanlıurfa          1101
Kocaeli             995
Mardin              990
Malatya             963
Erzurum             930
Samsun              929
Manisa              860
Nan                 791
Kahramanmaraş       762
Denizli             725
Trabzon             649
Sivas               640
Batman              619
Ağrı                605
Balıkesir           599
Ordu                561
Eskişehir           555
Adıyaman            534
Sakarya             520
Aydın               516
Tokat               509
Çorum               494
Zonguldak           448
Yozgat              433
Elazığ              430
Diğer               405
Şırnak              393
Tekirdağ            392
Muş  